In [1]:
from tensorflow.keras.datasets import mnist
(train_X, train_y), (test_X, test_y) = mnist.load_data()

train_X = train_X.reshape(-1, 28, 28, 1) / 255.0 
test_X = test_X.reshape(-1, 28, 28, 1) / 255.0 

print("train_X shape:", train_X.shape)
print("test_X shape:", test_X.shape)

train_X shape: (60000, 28, 28, 1)
test_X shape: (10000, 28, 28, 1)


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

model = Sequential()
model.add(Input(shape=(28,28,1)))
model.add(Conv2D(32, kernel_size=(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))

model.add(Conv2D(64, kernel_size=(3,3), activation="relu"))
model.add(MaxPooling2D()) # pool_size=(2,2)는 기본값 
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation="softmax"))

In [3]:
from tensorflow.keras.callbacks import ModelCheckpoint
filename = "model-digits.keras" 
checkpoint = ModelCheckpoint(filepath=filename,
                             monitor="val_accuracy", 
                             save_best_only=True, verbose=1)

In [4]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor="val_accuracy",  
                               patience=5, verbose=1)

In [5]:
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer="adam",
              metrics=["accuracy"])

In [6]:
model.fit(train_X, train_y, validation_data=(test_X, test_y),
          callbacks=[checkpoint, early_stopping], 
          batch_size=128, epochs=10)

Epoch 1/10
468/469 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.7909 - loss: 0.6499
Epoch 1: val_accuracy improved from -inf to 0.97880, saving model to model-digits.keras
469/469 ━━━━━━━━━━━━━━━━━━━━ 15s 27ms/step - accuracy: 0.7914 - loss: 0.6485 - val_accuracy: 0.9788 - val_loss: 0.0633
Epoch 2/10
467/469 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9648 - loss: 0.1142
Epoch 2: val_accuracy improved from 0.97880 to 0.98820, saving model to model-digits.keras
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - accuracy: 0.9648 - loss: 0.1142 - val_accuracy: 0.9882 - val_loss: 0.0371
Epoch 3/10
467/469 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9762 - loss: 0.0791
Epoch 3: val_accuracy improved from 0.98820 to 0.99020, saving model to model-digits.keras
469/469 ━━━━━━━━━━━━━━━━━━━━ 12s 24ms/step - accuracy: 0.9762 - loss: 0.0791 - val_accuracy: 0.9902 - val_loss: 0.0296
Epoch 4/10
467/469 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.9808 - loss: 0.0646
Epoch 4: val_accuracy did

In [7]:
model.evaluate(test_X, test_y)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9901 - loss: 0.0260


[0.020656615495681763, 0.9926000237464905]

In [8]:
from tensorflow.keras.models import load_model
model = load_model("model-digits.keras")

In [9]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

if cap.isOpened():
    while True:
        ret, img = cap.read()
        if not ret: break # 반환되는 프레임이 없으면 종료
            
        g_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        _, bin_img = cv2.threshold(g_img, 110, 255, cv2.THRESH_BINARY_INV)
        contours, hierarchy = cv2.findContours(bin_img,
                                               cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            (x,y), radius = cv2.minEnclosingCircle(contour)
            if radius < 5 : continue
            xs, xe = int(x-radius), int(x+radius)
            ys, ye = int(y-radius), int(y+radius)
            cv2.rectangle(bin_img, (xs,ys), (xe,ye), (200,0,0), 1)
            roi = bin_img[ys:ye, xs:xe]
            if roi.shape[0] * roi.shape[1] == 0:
                continue # ROI 배열의 행 또는 열의 크기가 0이면 resize시 예외 발생
            roi = cv2.resize(roi, dsize=(24,24))
            A = np.zeros((28,28))
            A[2:-2,2:-2] = roi
            A = A.reshape(-1,28,28,1)
            num = np.argmax(model.predict(A, verbose=0))
            cv2.putText(bin_img, str(num), (xs, ys), cv2.FONT_HERSHEY_PLAIN, 2, (200,0,0))
        cv2.imshow("Camera", bin_img)
        if cv2.waitKey(1) == 27: break # ESC
else :
    print("Camera not opened")
    
cap.release()
cv2.destroyAllWindows()

Camera not opened
